Let's experiment on the [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/). It includes 100,000 ratings from 1000 users on 1700 movies and was released in 1998. Find the zipfile [here](https://grouplens.org/datasets/movielens/100k/)

In [1]:
import pandas as pd
import zipfile
import os

In [2]:
target_folder = 'data/ml-100k'
with zipfile.ZipFile(target_folder + '.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
!ls 'data/ml-100k'

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


In [4]:
user_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user_df = pd.read_csv('data/ml-100k/u.user', sep='|', names=user_cols)
user_df.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
